In [1]:
# imports
import pandas as pd
import numpy as np
from collections import Counter
import re
import csv
import seaborn as sns
import matplotlib.pylab as plt

import gensim
from gensim.parsing.preprocessing import STOPWORDS

import warnings
warnings.filterwarnings('ignore')

# Import stopwords with nltk.
import nltk
from nltk.corpus import stopwords

STOP_WORD_FLAG = False

if STOP_WORD_FLAG:
    nltk.download("stopwords")
    stop = stopwords.words('english')
    STOP_WORD_FLAG = False

In [15]:
import datetime
import time
# stime = "Sat Aug 31 21:56:00 +0000 2019"
# print(datetime.datetime.strptime(stime, "%a %b %d %H:%M:%S %z %Y").timestamp())
#strftime('%d/%m/%Y') %X for 21:56:00

def date_parser(d):
    format_in =  "%a %b %d %H:%M:%S %z %Y"
    format_out = "%d/%m/%Y %X"
 
    d = datetime.datetime.strptime(d, format_in)
    return d.strftime(format_out)

date_parser("Sat Aug 31 21:56:00 +0000 2019")

'31/08/2019 21:56:00'

In [34]:
#Read the data
DATA_DIR = '../Data/'
source = pd.read_csv(DATA_DIR+'2019-08-31_tweetsevenmorebasic.csv',header=0, usecols=[1,2,5],
                     parse_dates=['created_at'],date_parser=date_parser)


In [3]:
# LATER

# source1.sample(10)

# retweet = re.compile(r'@([^\s:]+)')
# link = re.compile(r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')

# source["rt"] = source1.text.apply(lambda x:retweet.findall(x))
# source["link"] = source1.text.apply(lambda x:link.findall(x))


In [41]:
import preprocessor as p
from preprocessor.api import clean, tokenize, parse

source["clean_text"] = source.text.apply(lambda x: clean(x))
print(source.text[0])
print(source.clean_text[0])
print(tokenize(source.clean_text[0]))
print(tokenize(source.clean_text[0][0]))

RT @FrankSunTimes: Available in all good stores tomorrow... https://t.co/xPFIqu2zz0
: Available in all good stores tomorrow...
: Available in all good stores tomorrow...
:


In [37]:
source["parsed"] = source.text.apply(lambda x: parse(x))
source["mentions"] = source.parsed.apply(lambda x: x.mentions[0].match if x.mentions is not None else None)
source["urls"] = source.parsed.apply(lambda x: x.urls[0].match if x.urls is not None else None)
source["hashtags"]= source.parsed.apply(lambda x: x.hashtags[0].match if x.hashtags is not None else None)
source["reserveds"]= source.parsed.apply(lambda x: x.reserved[0].match if x.reserved is not None else None)

In [39]:
# source = source.drop(["id","text"], axis = 1)
source.head(5)

,author_id,created_at,text,clean_text,parsed,mentions,urls,hashtags,reserveds
0,619991287,2019-08-31 21:56:00,RT @FrankSunTimes: Available in all good store...,: Available in all good stores tomorrow...,<preprocessor.parse.ParseResult object at 0x7f...,@FrankSunTimes,https://t.co/xPFIqu2zz0,None,RT
1,2390646763,2019-08-31 21:56:00,VIDEO: Anna Wintour Makes This Year's Met Gala...,VIDEO: Anna Wintour Makes This Year's Met Gala...,<preprocessor.parse.ParseResult object at 0x7f...,None,https://t.co/sGIElNvKGK,None,None
2,27626694,2019-08-31 21:56:00,RT @NetoBrightman: @TheQuartering moments befo...,: moments before her twitter was deleted,<preprocessor.parse.ParseResult object at 0x7f...,@NetoBrightman,https://t.co/L7sppTVIrW,None,RT
3,29722915,2019-08-31 21:56:00,My sister and my baby brother are having a sle...,My sister and my baby brother are having a sle...,<preprocessor.parse.ParseResult object at 0x7f...,None,None,None,None
4,2864151534,2019-08-31 21:56:00,RT @vibesvt_: hoshi and eunhyuk attending each...,: hoshi and eunhyuk attending each other's con...,<preprocessor.parse.ParseResult object at 0x7f...,@vibesvt_,https://t.co/…,None,RT
